<a href="https://colab.research.google.com/github/SalimBobo/CapstoneProject1/blob/main/SalimBoBo_AviaTicket_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/content/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/content/train_data.csv
/content/test_data.csv
/content/.config/default_configs.db
/content/.config/active_config
/content/.config/.last_survey_prompt.yaml
/content/.config/config_sentinel
/content/.config/.last_update_check.json
/content/.config/gce
/content/.config/.last_opt_in_prompt.yaml
/content/.config/logs/2024.04.25/13.24.39.060363.log
/content/.config/logs/2024.04.25/13.24.09.661148.log
/content/.config/logs/2024.04.25/13.25.23.751786.log
/content/.config/logs/2024.04.25/13.25.24.520119.log
/content/.config/logs/2024.04.25/13.24.52.912147.log
/content/.config/logs/2024.04.25/13.25.09.448224.log
/content/.config/configurations/config_default


In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Ma'lumotlarni oldindan qayta ishlash funksiyasi
def preprocess_data(df):
    city_dic = {'Mumbai':1, 'Delhi':2, 'Bangalore':3, 'Kolkata':4, 'Hyderabad':5, 'Chennai':6}
    df['sorce_new_cities'] = [city_dic[i] for i in df['source_city']]
    df['destination_new_cities'] = [city_dic[i] for i in df['destination_city']]
    df.drop('flight', axis=1, inplace=True)
    df_adder = pd.get_dummies(df[['class', 'stops', 'airline', 'arrival_time', 'departure_time']], dtype='int')
    df = pd.concat([df, df_adder], axis=1)
    df.drop(columns=['airline', 'source_city', 'departure_time', 'stops', 'arrival_time',
       'destination_city', 'class'], inplace=True)
    return df

# Ma'lumotlarni yuklash
df_train = pd.read_csv("/content/train_data.csv")
df_test = pd.read_csv("/content/test_data.csv")

# Ma'lumotlarni oldindan qayta ishlash
df_train = preprocess_data(df_train)
df_test = preprocess_data(df_test)

# Ajratish
X = df_train.drop('price', axis=1)
Y = df_train['price'].copy()

# Xususiyatlarni scalelash
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train model
forest_model = RandomForestRegressor(n_estimators=1000)
forest_model.fit(X_scaled, Y)

# Test datani tayyorlash
Test_scaled = scaler.transform(df_test)

# Bashorat qilish
test_predictions = forest_model.predict(Test_scaled)

# Prediction ni CSV fayliga saqlash
submission = pd.DataFrame(df_test['id'])
submission['price'] = test_predictions
submission.to_csv('/content/submission.csv', index=False)